# Simulate Double Pendulum


In [1]:
using RigidBodyDynamics
using StaticArrays
using SymPy

┌ Info: Precompiling SymPy [24249f21-da20-56a4-8eb1-6a02cf4ae2e6]
└ @ Base loading.jl:1273


In [2]:
inertias = @syms m_1 m_2 I_1 I_2 positive = true
lengths = @syms l_1 l_2 c_1 c_2 real = true
gravitational_acceleration = @syms g real = true
params = [inertias..., lengths..., gravitational_acceleration...]
transpose(params)

1×9 LinearAlgebra.Transpose{Sym,Array{Sym,1}}:
 m_1  m_2  I_1  I_2  l_1  l_2  c_1  c_2  g

In [3]:
T = Sym # the 'scalar type' of the Mechanism we'll construct
axis = SVector(zero(T), one(T), zero(T)) # axis of rotation for each of the joints
double_pendulum = Mechanism(RigidBody{T}("world"); gravity = SVector(zero(T), zero(T), g))
world = root_body(double_pendulum) # the fixed 'world' rigid body

RigidBody: "world"

In [4]:
inertia1 = SpatialInertia(CartesianFrame3D("upper_link"),
    moment=I_1 * axis * transpose(axis),
    com=SVector(zero(T), zero(T), c_1),
    mass=m_1)
body1 = RigidBody(inertia1)
joint1 = Joint("shoulder", Revolute(axis))
joint1_to_world = one(Transform3D{T}, frame_before(joint1), default_frame(world));
attach!(double_pendulum, world, body1, joint1,
    joint_pose = joint1_to_world);

inertia2 = SpatialInertia(CartesianFrame3D("lower_link"),
    moment=I_2 * axis * transpose(axis),
    com=SVector(zero(T), zero(T), c_2),
    mass=m_2)
body2 = RigidBody(inertia2)
joint2 = Joint("elbow", Revolute(axis))
joint2_to_body1 = Transform3D(
    frame_before(joint2), default_frame(body1), SVector(zero(T), zero(T), l_1))
attach!(double_pendulum, body1, body2, joint2,
    joint_pose = joint2_to_body1)

Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow
No non-tree joints.

In [21]:
x = MechanismState(double_pendulum);
simplify.(transpose([x.q..., x.v...]))

1×4 Array{Sym,2}:
 0  0  0  0

In [23]:
q = configuration(x)
for i in eachindex(q)
    q[i] = symbols("q_$i", real = true)
end
simplify.(q)

2-element Array{Sym,1}:
 q_1
 q_2

In [24]:
v = velocity(x)
for i in eachindex(v)
    v[i] = symbols("v_$i", real = true)
end
simplify.(v)

2-element Array{Sym,1}:
 v_1
 v_2

In [9]:
simplify.(mass_matrix(x))

2×2 Array{Sym,2}:
 I_1 + I_2 + 2*c_2*l_1*m_2*cos(q_2) + l_1^2*m_2  I_2 + c_2*l_1*m_2*cos(q_2)
                     I_2 + c_2*l_1*m_2*cos(q_2)                         I_2

In [10]:
simplify(kinetic_energy(x))

     2        2                   2                                           
I₁⋅v₁    I₂⋅v₁               I₂⋅v₂               2                            
────── + ────── + I₂⋅v₁⋅v₂ + ────── + c₂⋅l₁⋅m₂⋅v₁ ⋅cos(q₂) + c₂⋅l₁⋅m₂⋅v₁⋅v₂⋅co
  2        2                   2                                              

          2      2
        l₁ ⋅m₂⋅v₁ 
s(q₂) + ──────────
            2     

In [11]:
simplify(gravitational_potential_energy(x))

-g⋅(c₁⋅m₁⋅cos(q₁) + c₂⋅m₂⋅cos(q₁ + q₂) + l₁⋅m₂⋅cos(q₁))